In [1]:
import pandas as pd
import json
import config

In [ ]:
def get_position_name(position: int):
    match position:
        case 1:
            return "GKP"
        case 2:
            return "DEF"
        case 3:
            return "MID"
        case 4:
            return "FWD"

In [ ]:
RAW_FILENAME = f"data/raw/player_stats/fpl_stats_{config.CURRENT_DATE}.json"

In [ ]:
with open(RAW_FILENAME,"r",encoding="utf-8") as f: # Source: https://fantasy.premierleague.com/api/bootstrap-static/
    all_data_raw = json.load(f)

In [ ]:
all_data = pd.DataFrame(all_data_raw["elements"])

Filter to only the columns we need

In [ ]:
allowed_cols = ["first_name","second_name","now_cost","ict_index","total_points","points_per_game","element_type","team_code","form"]
player_data = all_data[allowed_cols]

In [ ]:
player_data["position"] = player_data["element_type"].apply(lambda x: get_position_name(x))
player_data = player_data.drop(columns=["element_type"])
player_data = player_data.rename(columns={"now_cost":"cost"})

/tmp/ipykernel_259356/1572019178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["position"] = player_data["element_type"].apply(lambda x: get_position_name(x))


In [ ]:
#player_data = player_data.loc[all_data["status"]=="a"]

In [ ]:
player_data.head()

,first_name,second_name,cost,ict_index,total_points,points_per_game,team_code,form,position
0,Fábio,Ferreira Vieira,54,0.0,0,0.0,3,0.0,MID
1,Gabriel,Fernando de Jesus,68,11.7,6,0.7,3,0.7,FWD
2,Gabriel,dos Santos Magalhães,61,50.1,45,4.1,3,1.7,DEF
3,Kai,Havertz,80,73.1,46,4.2,3,1.3,FWD
4,Karl,Hein,40,0.0,0,0.0,3,0.0,GKP


In [ ]:
team_data = pd.DataFrame(all_data_raw["teams"])

In [ ]:
team_data = team_data[["code","short_name"]]

In [ ]:
team_data

,code,short_name
0,3,ARS
1,7,AVL
2,91,BOU
3,94,BRE
4,36,BHA
5,8,CHE
6,31,CRY
7,11,EVE
8,54,FUL
9,40,IPS


In [ ]:
def team_from_code(team_code):
    return team_data.loc[team_data["code"]==team_code]["short_name"].values[0]

In [ ]:
player_data["team"] = player_data["team_code"].apply(lambda x: team_from_code(x))

In [ ]:
player_data = player_data.drop(columns=["team_code"])

In [ ]:
player_data["first_name"] = player_data["first_name"] + " " + player_data["second_name"]
player_data = player_data.drop(columns=["second_name"])
player_data = player_data.rename(columns={"first_name": "name"})

In [ ]:
player_data.head()

,name,cost,ict_index,total_points,points_per_game,form,position,team
0,Fábio Ferreira Vieira,54,0.0,0,0.0,0.0,MID,ARS
1,Gabriel Fernando de Jesus,68,11.7,6,0.7,0.7,FWD,ARS
2,Gabriel dos Santos Magalhães,61,50.1,45,4.1,1.7,DEF,ARS
3,Kai Havertz,80,73.1,46,4.2,1.3,FWD,ARS
4,Karl Hein,40,0.0,0,0.0,0.0,GKP,ARS


In [ ]:
player_data.to_csv("./data/data.csv",index_label="id")